In [1]:
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
from tqdm import tqdm
import urllib.request

In [2]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [3]:
# urllib.request.urlretrieve(
#     "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
#     filename="ChatBotData.csv",
# )
Chatbot_Data = pd.read_csv("ChatBotData.csv") # 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
text = pd.read_csv('./data/새 폴더/KETI_대화데이터_일상_오피스.txt', delimiter='	', header=None)
text1 = text[text[0] == 1].drop(0, axis=1)
text2 = text[text[0] == 2].drop(0, axis=1)
text1.columns = ['Q']
text2.columns = ['A']
text = pd.concat([text1.reset_index(drop=True),text2.reset_index(drop=True)], axis=1)
text['label'] = 0
text

,Q,A,label
0,좋은 아침.,안녕하세요.,0
1,좋은 아침.,반갑습니다.,0
2,좋은 아침.,좋은 아침이에요.,0
3,좋은 아침.,간밤에 별일 없으셨죠?,0
4,좋은 아침.,안녕하시렵니까?,0
...,...,...,...
1320,눈 오는것 같은데?,"네, 눈길 조심하세요.",0
1321,너는 오늘 기분 어때?,기분 좋은 하루에요.,0
1322,무슨 생각해?,아무 생각도 안하는걸요.,0
1323,넌 뭐야?,잘 모르겠어요.,0


In [5]:
Chatbot_Data = pd.concat([Chatbot_Data, text], axis=0)
Chatbot_Data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
1320,눈 오는것 같은데?,"네, 눈길 조심하세요.",0
1321,너는 오늘 기분 어때?,기분 좋은 하루에요.,0
1322,무슨 생각해?,아무 생각도 안하는걸요.,0
1323,넌 뭐야?,잘 모르겠어요.,0


# 2회차 데이터 추가

In [6]:
daily1 = pd.read_json('./data/새 폴더/output_ daily_1st.json')
daily2 = pd.read_json('./data/새 폴더/output_daily_2nd.json')
daily3 = pd.read_json('./data/새 폴더/output_daily_3rd.json')

In [7]:
dailys = [daily1, daily2, daily3]

In [8]:
for i in range(len(dailys)):
    dailys[i] = dailys[i][['user_utterance','system_utterance']]
    dailys[i] = dailys[i][dailys[i].user_utterance != 'null'].reset_index(drop=True)
    dailys[i].columns = ['Q', 'A']
    dailys[i]['label'] = 0
daily = pd.concat(dailys)
daily

,Q,A,label
0,알고 있어.,오늘도 고생 많이 하셨습니다.,0
1,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.,0
2,알았다.,오늘도 고생이 많으셨습니다.,0
3,퇴근한다.,오늘도 일하느라 수고하셨어요.,0
4,알면 됐다.,그럼 내일 뵙겠습니다.,0
...,...,...,...
1990,그렇다.,곧 미팅이 시작됩니다. 잠시만 기다려주세요.,0
1991,그러하다.,오랜만에 오셨군요. 환영합니다.,0
1992,환영해주어서 감사해.,미팅이 곧 시작됩니다. 잠시만 기다려주세요.,0
1993,맞아.,어서오십시오. 오랜만입니다.,0


In [9]:
Chatbot_Data = pd.concat([Chatbot_Data, daily])
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [10]:
Chatbot_Data.shape

(26797, 3)

# 3회차 데이터 추가

In [10]:
# 압축파일 해제없이 내부 파일 읽기
from io import BytesIO
from zipfile import ZipFile
import json
import glob
import os

all_data = []
for i in glob.glob(os.getcwd()+'\\data\\01.데이터\\1.Training\\라벨링데이터\\*.zip'):
    data_list = []
    with ZipFile(i, 'r') as zipObj:
        listOfFileNames = zipObj.namelist()
        for fileName in listOfFileNames:
            zipRead = zipObj.read(fileName)
            json_data = zipRead.decode('utf-8')
            try:
                data=[]
                z = json.loads(json_data) # 여기서 오류가 나오는 경우가 있으므로 try문 적용
                chat = z['info'][0]['annotations']
                if chat['speaker_type'] == '1:1':
                    for i in chat['lines']:
                        data.append(i['norm_text'].replace('키키','').strip())
                if len(data) > 0:
                    data_list.append(data)
            except:
                pass
    all_data.append(data_list)

In [34]:
print(len(all_data))
print(len(all_data[0]))
print(len(all_data[1]))
print(len(all_data[0][0]))

8
12334
13338
17


In [11]:
labeling_data = []
for j in all_data:
    df_list = []
    for i in j:
        if len(i)%2 == 1:
            i = i[:-1]
        df = pd.DataFrame([i[::2], i[1::2]]).T
        df_list.append(df)
    df_list = pd.concat(df_list)
    labeling_data.append(df_list)
labeling_data = pd.concat(labeling_data)
labeling_data.columns = ['Q','A']
labeling_data['label'] = 0

In [37]:
labeling_data.head()

,Q,A,label
0,오빠야 저녁 식사는 뭐야?,나는 아직 뭐 먹을지 모르겠네 니는?,0
1,난 일단 지금 라면 먹고 있는데 저녁 모르겠어,오 지금 라면을 먹음 저녁 먹을 수 있나,0
2,모르겠어 이제 일어나서,어? 이제 일어났다고? 어?,0
3,아까 잠깐 깼다가 또 잤다,우와... 대단하다 언제 잤길래,0
4,새벽 5시? 많이 잔 것도 아님,그렇게 따지고 보면 그렇네?,0


In [12]:
Chatbot_Data = pd.concat([Chatbot_Data, labeling_data])

In [13]:
Chatbot_Data.shape

(497756, 3)

In [14]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [15]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 1,2회차는 epoch 30회
epoch_num = 10
Sneg = -1e18

In [16]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch, pin_memory=True)
# pin_memory를 이용하여 데이터를 고정된 주소에 로드
# 로컬에서는 큰 차이가 없었음;;

In [18]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

https://jaeyung1001.tistory.com/entry/bf16-fp16-fp32%EC%9D%98-%EC%B0%A8%EC%9D%B4%EC%A0%90

In [19]:
# FP16(반정밀도) 연산 적용하여 학습 속도 향상(약2분30초 -> 약1분 30초)
from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

for epoch in tqdm(range(epoch_num)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)
        
        with autocast():
            out = model(token_ids)
            out = out.logits
            mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
            mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
            loss = criterion(mask_out.transpose(2, 1), label)
            avg_loss = loss.sum() / mask.sum()
        
        scaler.scale(avg_loss).backward()
        scaler.step(optimizer)
        scaler.update()


  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\fkfma\AppData\Local\Temp\ipykernel_21168\2495599640.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
c:\Users\fkfma\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:650: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 10/10 

In [ ]:
# from torch.cuda.amp import GradScaler, autocast

# scaler = GradScaler()

# for epoch in tqdm(range(epoch_num)):
#     total_loss = 0.0
#     for batch_idx, samples in enumerate(train_dataloader):
#         optimizer.zero_grad()
#         token_ids, mask, label = samples
#         token_ids = token_ids.to(device)
#         mask = mask.to(device)
#         label = label.to(device)
        
#         with autocast():
#             out = model(token_ids)
#             out = out.logits
#             mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
#             mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
#             loss = criterion(mask_out.transpose(2, 1), label)
#             avg_loss = loss.sum() / mask.sum()
        
#         scaler.scale(avg_loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
        
#         total_loss += avg_loss.item()
    
#     print(f'Epoch {epoch+1}/{epoch_num}, Loss: {total_loss/len(train_dataloader)}')


In [ ]:
print ("start")
for epoch in tqdm(range(epoch_num)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

In [20]:
import os
model_save_path = "chatbot_model_497756.pth"
torch.save(
    {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
    },
    model_save_path,
)

Model saved at: chatbot_model_497756.pth


# 바로 모델을 추론하는 경우 아래부터 진행

In [ ]:
# import torch
# from transformers import GPT2LMHeadModel
# from transformers import PreTrainedTokenizerFast

# import numpy as np
# import pandas as pd
# import torch
# from torch.utils.data import DataLoader, Dataset
# from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
# import re
# from tqdm import tqdm
# import urllib.request

# Q_TKN = "<usr>"
# A_TKN = "<sys>"
# BOS = '</s>'
# EOS = '</s>'
# MASK = '<unused0>'
# SENT = '<unused1>'
# PAD = '<pad>'

# koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
#             bos_token=BOS, eos_token=EOS, unk_token='<unk>',
#             pad_token=PAD, mask_token=MASK) 
# model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# 모델 학습 완료 후 추론시 아래 코드부터 진행

In [3]:
model_path = "chatbot_model_497756.pth" 
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [5]:
# 기존 인코딩 코드 주석 내용
# sent='0'
# Q_TKN + q + SENT + sent + A_TKN + a

# 아래 두 코드는 학습없이 추론만 진행시 활성화
device = torch.device('cuda:0')
model.to(device)

with torch.no_grad():
    while True:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while True:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            input_ids = input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).to('cpu').squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        
        print("User > {}".format(q))
        print("Chatbot > {}".format(a.strip()))

c:\Users\fkfma\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:650: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


User > 너는 누구야?
Chatbot > 저는 위로해드리는 로봇이에요
User > 나를 위로해줘
Chatbot > 제가 위로해 드릴게요
User > 졸려
Chatbot > 낮잠을 잠깐 자도 괜찮아요
User > 지금 아침인데 자도 돼?
Chatbot > 응응 괜찮아
User > 존댓말 써
Chatbot > 존댓말 쓰면 좋지
User > 지금 대답이 안보여
Chatbot > 답은 언제나 내 안에 있죠
User > 아니 너가 존댓말 해야된다고
Chatbot > 아니
User > 너 몇살인데
Chatbot > 나랑 비슷해


In [ ]:
# GPT   temperature 0~1   1: 존댓말 및 따뜻한 표현    0: 차갑고 기계적인 대화